In [50]:
import math
from numpy import array, floor
from pandas import read_csv, DataFrame, concat
from itertools import product

In [72]:
root_path = 'C:/Users/shih/Project/'
def HRSs(year):
    return read_csv(f'{root_path}data/HRS_{year}.csv')

health_status_tran = read_csv(f'{root_path}transition prob/Health_Status_Transition_total.csv')
household_size_tran = read_csv(f'{root_path}transition prob/Household_Size_Transition_total.csv')

## Constants

In [3]:
r = 0.026
IOTA_M = 0.017
ALPHA = 0.35
DELTA = 0.017
KAPPA = 0.066
BETA = 0.906
ETA = 0.762
SIGMA = 2.006
GAMMA = 20.534
ZETA = 7619
c_floor = 13919

# By Appendix
r_h = DELTA + r + IOTA_M

def PSI(s):
    return 1 if s == 1 else 1.34

def CHI_s(s):
    return 1 if s == 1 else 1.48

def OMEGA(o):
    return 1 if o == 0 else 4.918

In [70]:
MAX_AGE = 95
s_set = {1, 2}
m_set = {1, 2, 3}
# x_set = range()
# p_set = range()

bins = array([28000, 72500, 118000, 160000, 200000, 245000, 301000, 400000, 682000])
def HouseAfford(money):
    maps = lambda x: x < money
    temp = maps(bins)
    return sum(temp)

## Functions

In [81]:
'''
Assumption:
Renter would never buy house
Pension income is constant during life cycle
Rents are constant
Housleholds only invest in risk-free bonds whose return is r
'''
def UtilityofRenter(i, b, s, m, x, p, a, k=0, h=0):
    if i == MAX_AGE:
        return 0
    else:
        max_u = 0
        largest_house_afford = HouseAfford(int(floor(((1+r)*a+b*CHI_s(s)-x)/r_h)))
        for h_index in range(0, largest_house_afford+1):
            h_tilde = bins[h_index]
            for a_prime in range(0, int(floor((1+r)*a+b*CHI_s(s)-x-r_h*h_tilde)), 1000):
                for c_tilde in range(0, int(floor((1+r)*a+b*CHI_s(s)-x-r_h*h_tilde-a_prime)), 1000):
                    c = max(c_floor * CHI_s(s) - r_h * h_tilde, c_tilde) if a_prime == 0 else c_tilde
                    # calculation
                    temp = 0
                    for s_prime in s_set:
                        index = (household_size_tran['age'] == i) & (household_size_tran['from'] == s) & (household_size_tran['to'] == s_prime)
                        pi_issp = household_size_tran[index]['p'].values[0]
                        for m_prime in m_set:
                            index = (health_status_tran['age'] == i) & (health_status_tran['from'] == m) & (health_status_tran['to'] == m_prime)
                            pi_immp = health_status_tran[index]['p'].values[0]
                            # for x_prime in x_set:
                            #     for p_prime in p_set:
                            x_prime = x
                            p_prime = p
                            temp = temp + pi_issp * pi_immp * UtilityofRenter(i+1, b, s_prime, m_prime, x_prime, p_prime, a_prime)
                    index = (health_status_tran['age'] == i) & (health_status_tran['from'] == m) & (health_status_tran['to'] == 4)
                    pi_im0 = health_status_tran[index]['p'].values[0]
                    utility = u(s, c, h, 0) + BETA * temp + BETA * pi_im0 * v(a_prime)
                    if temp > max_u:
                        max_u = temp
        return max_u

def u(s, c, h, o):
    try:
        return pow(pow(c, ETA)*pow(OMEGA(0)*h, 1-ETA)/PSI(s), 1-SIGMA)/(1-SIGMA)
    except:
        return 0

def v(a):
    try:
        return GAMMA * pow(a+ZETA, 1-SIGMA)/(1-SIGMA)
    except:
        return 0

In [15]:
data = HRSs(1996)
data['id + pn'] = data.apply(lambda x: x['id'] + x['pn'], axis=1)
data = data.drop(['id', 'pn'], axis=1)
for year in range(1998, 2022, 2):
    new = HRSs(year)
    new['id + pn'] = new.apply(lambda x: x['id'] + x['pn'], axis=1)
    new = new.drop(['id', 'pn'], axis=1)
    data = data.merge(new, how='inner', on='id + pn')
print(data)

         age_1996 gender_1996  hh_size_1996  hl_sta_1996 cnhMove_1996  \
0            40.0      Female             2          2.0            n   
1            50.0      Female             2          1.0            n   
2            49.0      Female             1          1.0            n   
3            60.0        Male             2          1.0            n   
4            53.0      Female             2          1.0            n   
...           ...         ...           ...          ...          ...   
2337525      57.0      Female             2          2.0            n   
2337526      68.0        Male             2          2.0            n   
2337527      54.0      Female             2          1.0            n   
2337528      57.0        Male             2          2.0            n   
2337529      49.0      Female             2          1.0            n   

         mdc_expen_1996  income_1996  fs_hold_1996  h_price_1996  \
0                   NaN          0.0           NaN     

In [41]:
index = (data['age_1996'] == 73)
data_point = data.loc[80617]['h_price_1996']
print(data_point)

nan


In [82]:
print(UtilityofRenter(94, 860, 1, 2, 20, 120000, 15000))

C:\Users\shih\AppData\Local\Temp\ipykernel_13748\3373509875.py:41: RuntimeWarning: divide by zero encountered in double_scalars
  return pow(pow(c, ETA)*pow(OMEGA(0)*h, 1-ETA)/PSI(s), 1-SIGMA)/(1-SIGMA)


0
